# Prepare SNP sets for GEDmatch

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import socket as socket
import os as os
import sys as sys
import h5py
import matplotlib.cm as cm
import itertools as it
import multiprocessing as mp

socket_name = socket.gethostname()
print(socket_name)

    
if socket_name.startswith("bionc"):
    print("Leipzig Cluster detected!")
    path = "/mnt/archgen/users/hringbauer/brienzi/"
    #sys.path.append("/mnt/archgen/users/hringbauer/data/malta/") 
    
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")
    
os.chdir(path)  # Set the right Path (in line with Atom default)
print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")

bionc21
Leipzig Cluster detected!
/mnt/archgen/users/hringbauer/brienzi
CPU Count: 40


In [56]:
def extract_23andme_snps(path_23, savepath=""):
    """Extract 23andme SNPs and save .tsv.
    path_23: 23andme .txt file
    savepath: Path to save output .tsv to"""
    df = pd.read_csv(path_23, comment='#', sep="\t", header=None, low_memory=False)
    df.columns = ["rsid","chr","pos", "gt"]
    if len(savepath)>0:
        df.to_csv(savepath, sep="\t", index=False)
        print(f"Saved {len(df)} SNPs to: {savepath}")
    return df

def create_23andme_snpfile(path_23, save_folder="", chs=range(1,23)):
    """Create bcftools_ch files for 23andme SNPs extracted
    from 23andme file.
    path_23: 23andme file
    save_folder: Where to save output to"""
    dft = extract_23andme_snps(path_23 = path_23, savepath ="")
    create_23andme_snpfile_fromdf(dft, save_folder=save_folder, chs=chs)

        
def create_23andme_snpfile_fromdf(df, save_folder="", chs=range(1,23)):
    """Create 23andme SNP file at save_folder from SNP dataframe df."""
    for ch in chs:
        df_ch = df[df["chr"]==str(ch)]
        df_save = df_ch[["chr","pos"]].astype("int32")

        savepath = os.path.join(save_folder, f"bcftools_ch{ch}.csv")
        df_save.to_csv(savepath, sep="\t", header=None, index=False)
        print(f"Saved {len(df_save)} SNPs to: {savepath}")
        
def overlap_with1240k(df, ch=1):
    """Calculate and print overlap 1240k and target SNPs in df,
    on chromosome ch"""
    marker_path = f"/mnt/archgen/users/hringbauer/data/1240kSNPs/bcftools_ch{ch}.csv"
    dft = pd.read_csv(marker_path, sep="\t", header=None)
    dft.columns = ["chr", "pos"]
    df_ch = df[df["chr"]==str(ch)]
    dfm = pd.merge(df_ch, dft, on=["pos"])
    print(f"Merged to {len(dfm)}/ {len(df_ch)} (23andme) & {len(dft)} (1240k) SNPs")
    
def create_23andme_snpfile_v45(path_v4, path_v5, save_folder="", 
                               snp_file_out="", chs=range(1,23)):
    """Create bcftools_ch files for 23andme SNPs extracted
    from 23andme file.
    path_23: 23andme file
    save_folder: Where to save output to"""
    df4 = extract_23andme_snps(path_23 = path_v4)
    df5 = extract_23andme_snps(path_23 = path_v5)
    
    ### Combine SNPs
    df45 = pd.concat((df4,df5)) # Concatenate
    df45u = df45.drop_duplicates(subset="rsid")
    print(f"Combined to unique {len(df45u)} / {len(df45)} total SNPs") # Drop Duplicates
    df45u = df45u.sort_values(by=["chr", "pos"])
    
    if len(snp_file_out)>0:
        df45u.to_csv(snp_file_out, sep="\t", index=False)
        print(f"Saved {len(df45u)} SNPs to: {snp_file_out}")   
    create_23andme_snpfile_fromdf(df45u, save_folder=save_folder, chs=chs)

### 1) Load 23andme data (v4 Chip: Omni Express)

In [3]:
df = extract_23andme_snps(path_23 = "/mnt/archgen/users/hringbauer/git/gedmatch_prep/data/genome_Harald_Ringbauer_v4_Full_20181111230705.txt",
                     savepath ="/mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme/snps.tsv")

Saved 601895 SNPs to: /mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme/snps.tsv


In [ ]:
df[df["chr"]=="1"]

### 1b) Load 23andme data (v5 Chip: GSA)

In [4]:
df5 = extract_23andme_snps(path_23 = "/mnt/archgen/users/hringbauer/git/gedmatch_prep/data/genome_Christopher_Smith_v5_Full_20230926164611.txt",
                           savepath ="/mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme/snps.v5.tsv")

Saved 632015 SNPs to: /mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme/snps.v5.tsv


In [28]:
dfm = pd.merge(df, df5, on="rsid")

In [42]:
print(f"v4 and v5 SNP overlap: {len(dfm)} / {len(df)} (v4) & {len(df5)} (v5)")

v4 and v5 SNP overlap: 116152 / 601895 (v4) & 632015 (v5)


# 2) [Optional] Check 23andme SNPs against 1240k
Specifically, for intersection with 1240k files
(to check whether indexing is same)

### v4 and 1240k Overlap

In [24]:
overlap_with1240k(df, ch=1)

Merged to 34517/ 46662 (23andme) & 93166 (1240k) SNPs


### v5 and 1240k Overlap

In [25]:
overlap_with1240k(df5, ch=1)

Merged to 10795/ 48932 (23andme) & 93166 (1240k) SNPs


# 3) Create 23andme SNP files in bed format

In [78]:
%%time

create_23andme_snpfile(path_23 = "/mnt/archgen/users/hringbauer/git/gedmatch_prep/data/genome_Harald_Ringbauer_v4_Full_20181111230705.txt",
                       save_folder = "/mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme")

Saved 46662 SNPs to: /mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme/bcftools_ch1.csv
Saved 46128 SNPs to: /mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme/bcftools_ch2.csv
Saved 38517 SNPs to: /mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme/bcftools_ch3.csv
Saved 33915 SNPs to: /mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme/bcftools_ch4.csv
Saved 34387 SNPs to: /mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme/bcftools_ch5.csv
Saved 40384 SNPs to: /mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme/bcftools_ch6.csv
Saved 33053 SNPs to: /mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme/bcftools_ch7.csv
Saved 30268 SNPs to: /mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme/bcftools_ch8.csv
Saved 26586 SNPs to: /mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme/bcftools_ch9.csv
Saved 29210 SNPs to: /mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme/bcftools_ch10.csv
Saved 293

### 3b) Create 23andme SNP files in bed format from v4 and v5 combined 
Also creates the 23andme SNP .tsv

In [57]:
%%time
create_23andme_snpfile_v45(path_v4 = "/mnt/archgen/users/hringbauer/git/gedmatch_prep/data/genome_Harald_Ringbauer_v4_Full_20181111230705.txt",
                           path_v5 = "/mnt/archgen/users/hringbauer/git/gedmatch_prep/data/genome_Christopher_Smith_v5_Full_20230926164611.txt",
                           snp_file_out = "/mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme/snps45.tsv",
                           save_folder="/mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme45", chs=range(1,23))

Combined to unique 1117758 / 1233910 total SNPs
Saved 1117758 SNPs to: /mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme/snps45.tsv
Saved 86953 SNPs to: /mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme45/bcftools_ch1.csv
Saved 89030 SNPs to: /mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme45/bcftools_ch2.csv
Saved 73967 SNPs to: /mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme45/bcftools_ch3.csv
Saved 66528 SNPs to: /mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme45/bcftools_ch4.csv
Saved 64891 SNPs to: /mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme45/bcftools_ch5.csv
Saved 75225 SNPs to: /mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme45/bcftools_ch6.csv
Saved 60961 SNPs to: /mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme45/bcftools_ch7.csv
Saved 56104 SNPs to: /mnt/archgen/users/hringbauer/git/gedmatch_prep/data/23andme45/bcftools_ch8.csv
Saved 47797 SNPs to: /mnt/archgen/users/hringbaue

# Area 51

## Explore 23andme File

In [90]:
df45u[df45u["rsid"]=="rs13304168"] #rs1236440 (U152) rs11799226 (L21) rs13304168 (L52) rs9785659 (ng) rs9786283 (ng) rs9786140 (L51)

,rsid,chr,pos,gt
595760,rs13304168,Y,14641193,C


In [64]:
df45u

,rsid,chr,pos,gt
0,rs548049170,1,69869,TT
1,rs9283150,1,565508,AA
2,rs116587930,1,727841,AG
0,rs12564807,1,734462,AA
1,rs3131972,1,752721,AG
...,...,...,...,...
596807,rs6568298,Y,59029728,T
596808,rs4047343,Y,59030922,G
596809,rs6568294,Y,59031514,--
596810,rs2334083,Y,59032331,C
